source: https://github.com/tensorflow/tensorflow/blob/master/tensorflow/contrib/distribute/python/examples/simple_tfkeras_example.py

In [1]:
import numpy as np
import tensorflow as tf

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [16]:
def input_fn():
    x = np.random.random((1024, 10))
    y = np.random.randint(2, size=(1024,1))
    x = tf.cast(x, tf.float32)
    
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.repeat(10)
    dataset = dataset.batch(32)
    
    return dataset


In [11]:
CHECKPOINTS = "./checkpoints"

In [13]:
%mkdir -p $CHECKPOINTS

In [5]:
# still not supported in tensorflow 1.7
#strategy = tf.contrib.distribute.MirroredStrategy(
#    ['/device:CPU:0',
#    #'/device:GPU:0',
#    #'/device:GPU:1',
#    ])
#config = tf.estimator.RunConfig(train_distributes=strategy)
config = None
optimizer = tf.train.GradientDescentOptimizer(0.2)

In [6]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(16, activation='relu', input_shape=(10,)))
model.add(tf.keras.layers.Dense(1,  activation='sigmoid'))

In [7]:
model.compile(loss='binary_crossentropy', optimizer=optimizer)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 16)                176       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 17        
Total params: 193
Trainable params: 193
Non-trainable params: 0
_________________________________________________________________


In [8]:
tf.keras.backend.set_learning_phase(True)

In [14]:
keras_estimator = tf.keras.estimator.model_to_estimator(
    keras_model=model,
    config=config,
    model_dir=CHECKPOINTS)

INFO:tensorflow:Using the Keras model provided.
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './checkpoints', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1c23503ba8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [17]:
keras_estimator.train(input_fn=input_fn,
                      steps=10)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./checkpoints/keras_model.ckpt
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 2 into ./checkpoints/model.ckpt.
INFO:tensorflow:loss = 0.7554135, step = 2
INFO:tensorflow:Saving checkpoints for 12 into ./checkpoints/model.ckpt.
INFO:tensorflow:Loss for final step: 0.7131709.


In [18]:
eval_result = keras_estimator.evaluate(input_fn=input_fn)
print('Eval result: {}'.format(eval_result))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-04-09-03:56:34
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./checkpoints/model.ckpt-12
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-04-09-03:56:35
INFO:tensorflow:Saving dict for global step 12: global_step = 12, loss = 0.6979753
Eval result: {'loss': 0.6979753, 'global_step': 12}
